## 0. Libarary 불러오기 및 경로설정

In [21]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import torch.optim as optim

In [16]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

## 1. Model 정의

In [3]:
# Conv_block
# activation = relu
# y = relu(BN(conv(x)))
class Conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, activation=True, **kwargs) -> None:
        super(Conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs) # kernel size = ...
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.activation = activation

    def forward(self, x):
        if not self.activation:
            return self.batchnorm(self.conv(x))
        return self.relu(self.batchnorm(self.conv(x)))

In [4]:
class Res_block(nn.Module):
    def __init__(self, in_channels, red_channels, out_channels, is_plain=False):
        super(Res_block,self).__init__()
        self.relu = nn.ReLU()
        self.is_plain = is_plain
        
        if in_channels==64:
            self.convseq = nn.Sequential(
                Conv_block(in_channels, red_channels, kernel_size=1, padding=0),
                Conv_block(red_channels, red_channels*2, kernel_size=3, padding=1),
                Conv_block(red_channels*2, out_channels, activation=False, kernel_size=1, padding=0)
            )
            self.iden = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        elif in_channels == out_channels:
            self.convseq = nn.Sequential(
                Conv_block(in_channels, red_channels, kernel_size=1, padding=0),
                Conv_block(red_channels, red_channels*2, kernel_size=3, padding=1),
                Conv_block(red_channels*2, out_channels, activation=False, kernel_size=1, padding=0)
            )
            self.iden = nn.Identity()
        else:
            self.convseq = nn.Sequential(
                Conv_block(in_channels, red_channels, kernel_size=1, padding=0, stride=2),
                Conv_block(red_channels, red_channels*2, kernel_size=3, padding=1),
                Conv_block(red_channels*2, out_channels, activation=False, kernel_size=1, padding=0)
                
            )
            self.iden = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2)
        
    def forward(self, x):
        y = self.convseq(x)
        if self.is_plain:
            x = y
        else:
            x = y + self.iden(x)
        x = self.relu(x)  # relu(skip connection)
        return x

In [8]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000, is_plain=False):
        super(MyModel, self).__init__()
        self.layers = []
        self.ksize = 3
        
        #L1
        self.conv1 = Conv_block(3,32,kernel_size=self.ksize, padding=self.ksize//2, stride=(1,1))
        self.conv2 = Conv_block(32,64,kernel_size=self.ksize, padding=self.ksize//2, stride=(2,2))
        #Res1 
        self.conv3_x = nn.Sequential(
            Res_block(64, 32, 64, is_plain)
        )
        #L2
        self.conv4 = Conv_block(64,128,kernel_size=self.ksize, padding=self.ksize//2, stride=(2,2))
        #Res2  
        self.conv5_x = nn.Sequential(
            Res_block(128, 64, 128, is_plain),
            Res_block(128, 64, 128, is_plain)
        )  
        #L3
        self.conv6 = Conv_block(128,256,kernel_size=self.ksize, padding=self.ksize//2, stride=(2,2))
        #Res3  
        self.conv7_x = nn.Sequential(
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
            Res_block(256, 128, 256, is_plain),
        )
        #L4
        self.conv8 = Conv_block(256,512,kernel_size=self.ksize, padding=self.ksize//2, stride=(2,2))
        #Res4
        self.conv9_x = nn.Sequential(
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain),
            Res_block(512, 256, 512, is_plain)
        )
        #L5
        self.conv10 = Conv_block(512,1024,kernel_size=self.ksize, padding=self.ksize//2, stride=(2,2))
        #Res5
        self.conv11_x = nn.Sequential(
            Res_block(1024, 512, 1024, is_plain),
            Res_block(1024, 512, 1024, is_plain),
            Res_block(1024, 512, 1024, is_plain),
            Res_block(1024, 512, 1024, is_plain),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.classifier = nn.Sequential(
            # nn.Dropout(),
            # nn.Linear(64, 32),
            # nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3_x(x)
        x = self.conv4(x)
        x = self.conv5_x(x)
        x = self.conv6(x)
        x = self.conv7_x(x)
        x = self.conv8(x)
        x = self.conv9_x(x)
        x = self.conv10(x)
        x = self.conv11_x(x)
        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2. Test Dataset 정의

In [51]:
class TrainDataset(Dataset):
    def __init__(self, train_idx, transform):
        self.train_idx = train_idx
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.train_idx[index][0])

        if self.transform:
            image = self.transform(image)
        return image, train_idx[index][1]

    def __len__(self):
        return len(self.train_idx)

# class TestDataset(Dataset):
#     def __init__(self, img_paths, transform):
#         self.img_paths = img_paths
#         self.transform = transform

#     def __getitem__(self, index):
#         image = Image.open(self.img_paths[index])

#         if self.transform:
#             image = self.transform(image)
#         return image

#     def __len__(self):
#         return len(self.img_paths)

### Trainning

In [39]:
# meta 데이터와 이미지 경로를 불러옵니다.
# 7 : 3 비율로 나눕니다.
# submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
# submission_train = submission[:int(len(submission)*0.7)]
# submission_test = submission[int(len(submission)*0.7):]

In [52]:

submission = pd.read_csv(os.path.join(train_dir, 'train_label.csv'))
image_dir = os.path.join(train_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
# image_paths = [os.path.join(image_dir, img_id) for img_id in submission.path]
train_idx = [i for i in zip(submission.path,submission.label)]
transform = transforms.Compose([
    transforms.CenterCrop(256),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TrainDataset(train_idx, transform)

loader = DataLoader(
    dataset,
    shuffle=True
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)

In [ ]:
#입력 크기 테스트
from torchsummary import summary
summary(model, input_size=(3, 256, 256), device=device.type) 

In [53]:
def func_eval(model,data_iter,device):
    with torch.no_grad():
        n_total,n_correct = 0,0
        model.eval() # evaluate (affects DropOut and BN)
        for image, target in data_iter:
            y_trgt = target.to(device)
            model_pred = model(image.view(-1,1,28,28).to(device))
            _,y_pred = torch.max(model_pred.data,1)
            n_correct += (y_pred==y_trgt).sum().item()
            n_total += image.size(0)
        val_accr = (n_correct/n_total)
        model.train() # back to train mode 
    return val_accr
print ("Done")

Done


In [54]:
next(iter(loader))

[tensor([[[[ 1.7157,  1.6569,  1.5980,  ...,  1.2059,  1.1863,  1.1471],
           [ 1.7941,  1.7745,  1.7157,  ...,  1.1667,  1.1667,  1.1275],
           [ 1.8922,  1.8725,  1.8333,  ...,  1.1275,  1.1275,  1.1078],
           ...,
           [-0.6569, -0.6961, -0.7549,  ..., -0.4216, -0.4216, -0.4020],
           [-0.6765, -0.7157, -0.7745,  ..., -0.4412, -0.4608, -0.5000],
           [-0.6765, -0.7157, -0.7941,  ..., -0.4412, -0.4804, -0.5588]],
 
          [[ 1.2255,  1.1667,  1.1078,  ...,  0.8725,  0.8529,  0.8137],
           [ 1.3039,  1.2843,  1.2255,  ...,  0.8333,  0.8333,  0.7941],
           [ 1.4020,  1.3824,  1.3431,  ...,  0.7941,  0.7941,  0.7745],
           ...,
           [-0.7549, -0.7941, -0.8529,  ..., -0.5000, -0.5000, -0.4804],
           [-0.7745, -0.8137, -0.8725,  ..., -0.5196, -0.5392, -0.5784],
           [-0.7745, -0.8137, -0.8922,  ..., -0.5196, -0.5588, -0.6373]],
 
          [[ 0.6176,  0.5588,  0.5000,  ...,  0.2843,  0.2647,  0.2255],
           [ 

In [55]:
print ("Start training.")
optm = optim.Adam(model.parameters(),lr=1e-3)
#optm = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
loss = nn.CrossEntropyLoss()
# model.init_param() # initialize parameters
model.train() # to train mode 
EPOCHS,print_every = 10,1
for epoch in range(EPOCHS):
    loss_val_sum = 0
    for images, target in loader:
        # Forward path
        y_pred = model.forward(images.to(device))
        loss_out = loss(y_pred,target.to(device))
        # Update
        optm.zero_grad()     # reset gradient 
        loss_out.backward()      # backpropagate
        optm.step()      # optimizer update
        loss_val_sum += loss_out
    loss_val_avg = loss_val_sum/len(loader)
    # Print
    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        train_accr = func_eval(model,loader,device)
        #test_accr = func_eval(model,test_iter,device)
        print ("epoch:[%d] loss:[%.3f] train_accr:[%.3f]"%
               (epoch,loss_val_avg,train_accr))
        # print ("epoch:[%d] loss:[%.3f] train_accr:[%.3f] test_accr:[%.3f]."%
        #        (epoch,loss_val_avg,train_accr,test_accr))
print ("Done")

Start training.


## 3. Inference

In [10]:


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             896
       BatchNorm2d-2         [-1, 32, 256, 256]              64
              ReLU-3         [-1, 32, 256, 256]               0
        Conv_block-4         [-1, 32, 256, 256]               0
            Conv2d-5         [-1, 64, 128, 128]          18,496
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
        Conv_block-8         [-1, 64, 128, 128]               0
            Conv2d-9         [-1, 32, 128, 128]           2,080
      BatchNorm2d-10         [-1, 32, 128, 128]              64
             ReLU-11         [-1, 32, 128, 128]               0
       Conv_block-12         [-1, 32, 128, 128]               0
           Conv2d-13         [-1, 64, 128, 128]          18,496
      BatchNorm2d-14         [-1, 64, 1